# Numpy internals

---

In this notebook I want to show you some of the internals of numpy arrays and the consequences for working with arrays.

---

After introducing `numpy`-arrays some informations may be irritating, e.g. what operations will create a copy of an array, which operations will create a view. This has direct consequences in how much memory your program is using during the process or how you can speed up the code.

## 1. Memory layout

To understand `numpy`-arrays it is necessary to understand how `numpy` is organizing the data. Let's start with a 2d-`numpy`-array:

In [ ]:
import numpy as np

a = (np.arange(1,13,1)*5).reshape((3,4))

print(a)

<center>
    <img src="figs/memory_layout.png" style="width:75%" />
    <br>Memory layout: Array representation (left), contiguous block memory (right)
</center>

In this representation (we assume C notation) all the data of all axes are stored linear in the computers memory. (For advanced users, `numpy` can store memory also in Fortran F notation, which is column orientatet. In this case it means, 5,25,45,10,30,50,... !)

For accessing the data `numpy` needs exactly 3 parameters:
 * pointer to the memory block
 * shape of the array representation
 * so called `strides`, which defines the distance between elements for each axis (jump values)

In [ ]:
print(a.__array_interface__)  # data the pointer
print(a.shape)                # shape of the array
print(a.strides)              # jump bytes for next elements
print(a.dtype)                # int64 => 8 bytes

Starting with the element `a[1,1] = 30` how we can jump to the elements `a[1,2] = 35` and `a[2,1] = 50`:
  *  `a[1,2]`: 1 element in the axis 1 (=> 8 bytes -> 1 element in the contiguous block)
  *  `a[2,1]`: 1 element in the axis 0 (=> 32 bytes -> 4 elements in the contiguous block)

For higher dimensional arrays, the data are similar organized. For a 3d-array all 2d-slices are stored linear one after the other. Striding works for indexing.

---

## 2. Reshaping

If you reshape the array, e.g. now in 4 rows with 3 column, only the metadata will change, the data is the same:

In [ ]:
b = a.reshape((4,3))
print(b)
print(b.__array_interface__)  # data the pointer
print(b.shape)                # shape of the array
print(b.strides)              # jump bytes for next elements

In [ ]:
c = a.reshape((12))
print(c)
print(c.__array_interface__)  # data the pointer
print(c.shape)                # shape of the array
print(c.strides)              # jump bytes for next elements

---

## 3. Slicing

Slicing may sounds difficult in handling, but in fact, you can mathematically represent all slicing operations with a change of the 3 parameter, `pointer`, `shape`, and `strides`:

In [ ]:
d = a[0:2,1:3]
print(d)

<center>
    <img src="figs/memory_layout2.png" style="width:75%" />
    <br>Memory layout after slicing: Array representation (left), contiguous block memory (right)
</center>

In [ ]:
print(a.__array_interface__)
print(a.shape)
print(a.strides)
print(d.__array_interface__)  # data the pointer
print(d.shape)                # shape of the array
print(d.strides)              # jump bytes for next elements

In this case the `strides` may be the same, but for this example:

In [ ]:
e = a[::2,::2]  # access every second column and row

print(e)

print(a.__array_interface__)
print(a.shape)
print(a.strides)
print(e.__array_interface__)  # data the pointer
print(e.shape)                # shape of the array
print(e.strides)              # jump bytes for next elements

---

## 4. Transpose

If you use the `.T` transpose function, you swap basically the two axes in the 2d example. This can be done with changing the parameters as well.

In [ ]:
f = a.T
print(f)
print(a.__array_interface__)
print(a.shape)
print(a.strides)
print(f.__array_interface__)  # data the pointer
print(f.shape)                # shape of the array
print(f.strides)         

(For advanced user, this is a nice example, to show, that also the F notation, as mentioned above, can be used without any problems, since all operations are working similar!)

---

## 5. Copy vs. view

As you have maybe briefly seen, after reshaping of arrays, the pointer to the data block has not changed. In this case, the new variable points to the same data block as before and is called `view`. 

Let's have a look at this more generally.

`numpy` is optimized in several ways, which means that one goal is not to use additional memory for operations if not necessary. This means of course that if a operation means to change only the meta data, e.g. `shape` and `strides` always a `view` will be created. If a changed `pointer` will point to the same memory region as before, also a `view` is created. 

We can simply prove this:

In [ ]:
import numpy as np

a = np.array([[1,2,3,4],[5,6,7,8]])
print(a)
print(a.base)   # if none, a is a real data block

b = a.reshape((4,2))
print(b)
print(b.base)   # points to the original data block

also the return values of slicing operations are `views`:

In [ ]:
c = a[1:,2:]
print(c)
print(c.base)

print(a.__array_interface__)
print(c.__array_interface__)  # data has changed

In this example, also the `pointer` has changed, but it is still a `view`.

---

## 6. Fancy indexing

Fancy indexing, indexing with index-arrays or book-arrays, is special to the numpy internals. In general there is no way to adjust the metadata parameters to fulfill the indexing wish on the same real data block. So in this case there is the only possibility to create a copy of the selected data block:

In [ ]:
import numpy as np

a = np.arange(1,13,1).reshape((3,4))

print(a)

b = a[np.array([1,2])]

print(b)
print(b.base)   # not a view anymore

**Note:** Whenever fancy indexing is in the game, a copy will be created, even if the fancy index is a slicing rule!

---

## 7. Left hand side operations

As presented in all the notebooks, with `numpy`-arrays all indexing/slicing operations can be used also on the left hand side operations, in assignments:

In [ ]:
import numpy as np

a = np.arange(1,13,1).reshape((3,4))
print(a)

a[1] = -100
a[2,1:3] = np.array([-1,-5])

print(a)

Applying the operations on views, you will get the same results:

In [ ]:
a = np.arange(1,13,1)

b = a.reshape((3,4))
print(b)

b[1] = -100
b[2,1:3] = np.array([-1,-5])

print(b)
print(a)  # elements are modified also!

To change all values at the same time, you need to *slice* with all elements:

In [ ]:
a = np.arange(1,13,1)

a[:] = -100
# obviously a = -100 is not working ;-)

Fancy indexing is in this case also special:

In [ ]:
a = np.arange(1,13,1)

mask = np.array([2,3,7,8,9])
b = a[mask]

print(b)
b[:] = -100      # change all values, 

print(b)      # working on a copy!
print(a)      # nothing has changed!

In this example, we only changed the copied data, if you need to modify the orginal data, you have to use this:

In [ ]:
a = np.arange(1,13,1)

mask = np.array([2,3,7,8,9])

a[mask] = -100

print(a)

In this case the left hand side object acts like a view.

---

## 8. `np.nonzero` handling

In several occasions in the exercises/projects I refer to a function `np.nonzero`. 

The technical function is very simple, `np.nonzero` returns the indices of all elements in an array which are `!=0`. For each axis of the array, an array with the indices will be returned. 

Usually with normal numbers, the function makes no real sense, but if you analyse fancy indexing masks with `np.nonzero` you will get indices of all `True`elements.

In [ ]:
import numpy as np

a = np.arange(0,12,1).reshape((3,4))

print(a)

mask = (a < 4) | (a > 8)

print(mask)

nz = np.nonzero(mask)
print(nz)

In this example we will get two numpy-arrays back for each axis. To understand what the indices mean, you can use these to address the elements in the main array:

In [ ]:
print(a[nz])            # applying the index arrays, both at the same time
# print(a[nz[0],nz[1]])

# equivalent to

print(a[mask])          # applying mask instead

This may quite helpful, if you select data points from an array with fancy indexing first, but needs to have indices for another operations.